## Trying OpenAlex for getting research papers
<!-- Papers
Id
Title
Abstract
TLDR
URL
Year
CitationCount
isOpenAccess
Embedding (optional if creating custom embedding)
Citations (other paper’s references) -->


In [ ]:
# Papers
# Id
# Title
# Abstract
# TLDR
# URL
# Year
# CitationCount
# isOpenAccess
# Embedding (optional if creating custom embedding)
# Citations (other paper’s references)


In [45]:
import requests
import json

def search_open_alex(query, offset=0, limit=10, fields=[]):
    base_url = "https://api.openalex.org/works"
    
    # URL parameters
    params = {
        'search': query,
        'per-page': 200, # max
    }
    
    # Send GET request
    response = requests.get(base_url, params=params)
    
    # Raise an exception if the GET request fails
    response.raise_for_status()
    
    return response.json()

# Call the function, fields found here: https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/search-entities
result = search_open_alex("carbon capture") 

# Open the file in append mode
with open('openalex/output.json', 'w') as file:
    print("result", len(result["results"]))
    json_string = json.dumps(result, indent=2)
    # json_string = preprocess_latent_lab(result['data'])
    file.write(json_string + '\n')

carbon_capture_results = result["results"]

result 200


In [2]:
from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders

In [3]:
import pyalex

pyalex.config.email = "1kevin.huang@gmail.com"

In [58]:
results_pager = Works().search("carbon capture").sort(cited_by_count="desc").paginate(per_page=200)


In [57]:
count = 0
for page in results_pager:
    count += 1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [66]:
top_results = []
for i, page in enumerate(results_pager):
    top_results += page
    print(len(page), len(top_results))
    if (i == 9):
        break

200 200
200 400
200 600
200 800
200 1000
200 1200
200 1400
200 1600
200 1800
200 2000


In [67]:
# Open the file in append mode
with open('openalex/raw_output.json', 'w') as file:
    json_string = json.dumps(top_results, indent=2)
    # json_string = preprocess_latent_lab(result['data'])
    file.write(json_string + '\n')

In [68]:
# Extracting the desired columns
import json

# Specify the output file path
output_file_path = "openalex/extracted_results.json"

# Create an empty list to store the extracted results
extracted_data = []

start_index = 0

# Check if the JSON file already exists, use this code if it stops halfway
# try:
#     with open(output_file_path, "r") as existing_file:
#         existing_data = json.load(existing_file)
#         num_existing_entries = len(existing_data)
# except FileNotFoundError:
#     num_existing_entries = 0

# # Print the number of existing entries and the index to start on
# print("Number of existing entries in the JSON file:", num_existing_entries)
# start_index = num_existing_entries
# print("Start index:", start_index)

# Iterate over the results and extract the desired columns
for i, result in enumerate(top_results[start_index]):
    print(i)
    try:
        extracted_result = {
            "id": result["id"],
            "doi": result["doi"],
            "title": result["display_name"],
            "isOpenAccess": result["primary_location"]["is_oa"],
            "abstract": Works()[result["id"].split("https://openalex.org/")[1]]["abstract"],
            # "authors": result.get("authorships", []),
            "url": result["primary_location"].get("landing_page_url", ""),
            "citations": result.get("cited_by_api_url", ""),
            "citationCount": result.get("cited_by_count", 0),
            "concepts": result.get("concepts", []),
            "year": result["publication_year"]
        }
        extracted_data.append(extracted_result)

        # Write the extracted result to the JSON file
        with open(output_file_path, "a") as output_file:
            json.dump(extracted_result, output_file)
            output_file.write("\n")  # Add a new line after each result

    except Exception as e:
        print(f"Error occurred for result {i}: {str(e)}")
        continue

# Print the number of retrieved entries
num_retrieved_entries = len(extracted_data)
print("Number of retrieved entries:", num_retrieved_entries)
print("Extraction results written to:", output_file_path)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# Add and compute SPECTRE embeddings for each paper title and abstract!


## Semantic Scholar

In [ ]:
import requests
import json

def search_semantic_scholar(query, offset=0, limit=10, fields=[]):
    base_url = "http://api.semanticscholar.org/graph/v1/paper/search"
    
    # URL parameters
    params = {
        'query': query,
        'offset': offset,
        'limit': limit,
        'fields': ','.join(fields),
    }
    
    # Send GET request
    response = requests.get(base_url, params=params)
    
    # Raise an exception if the GET request fails
    response.raise_for_status()
    
    return response.json()

# Call the function, fields found here: https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/post_graph_get_papers
result = search_semantic_scholar("carbon capture", offset=10, limit=350, fields=["title", "citationCount", "abstract", "url", "year", "isOpenAccess", "tldr", "citations", "embedding"]) 

# Convert to Latent Lab keys
def preprocess_latent_lab(data):
    for entry in data:
        if 'abstract' in entry:
            entry['description_cleaned'] = entry.pop('abstract')
        if 'year' in entry:
            entry['created'] = entry.pop('year')

    # If you need to convert it back to a JSON string:
    new_json_string = json.dumps(data, indent=2)
    return new_json_string

# Open the file in append mode
with open('output_1000.json', 'w') as file:
    json_string = json.dumps(result['data'], indent=2)
    # json_string = preprocess_latent_lab(result['data'])
    file.write(json_string + '\n')